# Import

In [1]:
from __future__ import division
import warnings
warnings.filterwarnings('ignore')

from numpy.random import randint
from numpy.random import rand
import random
import math
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tqdm.contrib.concurrent import process_map

# Algorithms

## PSO

In [2]:
class Particle:
    def __init__(self, x0,num_dimensions):
        self.num_dimensions = num_dimensions
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self, costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self, pos_best_g):
        w = 0.7       # constant inertia weight (how much to weigh the previous velocity)
        c1 = 1.4        # cognative constant
        c2 = 1.4        # social constant

        for i in range(0, self.num_dimensions):
            r1 = random.random()
            r2 = random.random()            
            vel_cognitive = c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social = c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i] = w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self, bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i] = self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]


def evaluation(particle):
    particle.evaluate(cost_function)
    return particle


class PSO:
    def __init__(self, costFunc, bounds, num_particles, maxiter, num_dimensions):
        err_best_g = -1
        pos_best_g = []        
        swarm = []
                
        swarm = []
        for i in range(0, num_particles):
            x0 = [random.uniform(bounds[j][0],bounds[j][1]) for j in range(num_dimensions)]
            swarm.append(Particle(x0, num_dimensions))            
        
        ############## main loop #######################
        for i in range(maxiter):                       
            counter = 0 
            for particle in process_map(evaluation, swarm, tqdm_class=tqdm, max_workers=22, chunksize=5):
                if particle.err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(particle.position_i)
                    err_best_g=float(particle.err_i)                
                swarm[counter] = particle
                counter += 1
            
            # cycle through swarm and update velocities and position                        
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)                        
            print(err_best_g)                        
            print(pos_best_g)
        # print final results        
        print(pos_best_g)
        print(err_best_g)

## GA

In [2]:
class Individual:
    def __init__(self, costFunc, bounds, num_dimensions, r_mut, new=True):
        self.num_dimensions = num_dimensions
        self.bounds = bounds    
        self.r_mut = r_mut
        self.costFunc = costFunc
        if(new):
            self.random_solution()
            self.evaluate()
    
    def copy(self):
        clone = Individual(self.costFunc, self.bounds, self.num_dimensions, self.r_mut, new=False)
        clone.position = self.position.copy()
        clone.cost = self.cost
        return clone
    
    def random_solution(self):
        self.position = [random.uniform(bound[0], bound[1]) for bound in self.bounds]        
        
    def evaluate(self):
        self.cost = self.costFunc(self.position)        
        
    def mutation(self):                
        for i in range(self.num_dimensions):
            if rand() < self.r_mut:        
                self.position[i] = random.uniform(self.bounds[i][0], self.bounds[i][1])                        

class Genetic_Algorithm:
    def create_individual(self, inp):
        return Individual(self.costFunc, self.bounds, self.num_dimensions, self.r_mut)
    
    def selection(self, scores, k=3):    
        selection_ix = randint(len(self.pop))        
        for ix in randint(0, len(self.pop), k-1):        
            if scores[ix] < scores[selection_ix]:
                selection_ix = ix
        return self.pop[selection_ix]
    
    def crossover_one_point(self, p1, p2):    
        c1 = p1.copy()
        c2 = p2.copy()            
        if rand() < self.r_cross:                 
            pt = randint(1, len(p1.position)-2)                    
            c1.position = p1.position[:pt] + p2.position[pt:]
            c2.position = p2.position[:pt] + p1.position[pt:]                
        return [c1, c2]
    
    def crossover_uniform(self, p1, p2):    
        c1 = p1.copy()
        c2 = p2.copy()            
        for i in range(len(p1.position)):
            if(rand() < 0.5):
                c1.position[i] = p2.position[i] 
                c2.position[i] = p1.position[i]                 
        return [c1, c2]
    
    def apply_operations(self, index):        
        p1 = self.selected[index]        
        p2 = self.selected[index+1]      
        if(rand() > 0.5):
            c1, c2 = self.crossover_one_point(p1, p2)                        
        else:
            c1, c2 = self.crossover_uniform(p1, p2)                        
        c1.mutation()
        c2.mutation()
        c1.evaluate()
        c2.evaluate()
        return [c1, c2]

    def __init__(self, costFunc, bounds, num_individual, maxiter, num_dimensions, r_cross, r_mut):                    
        self.costFunc = costFunc
        self.bounds = bounds
        self.num_individual = num_individual
        self.maxiter = maxiter
        self.num_dimensions = num_dimensions
        self.r_cross = r_cross
        self.r_mut = r_mut
        
        best = self.create_individual(None)
        
        self.pop = []
        for individual in process_map(self.create_individual, range(0,num_individual), tqdm_class=tqdm, max_workers=22, chunksize=5):
            self.pop.append(individual)                        
    
        for gen in tqdm(range(maxiter)):
            scores = [individual.cost for individual in self.pop]        
            for individual in self.pop:
                if individual.cost < best.cost:
                    best = individual.copy()
                    
            self.selected = [self.selection(scores) for _ in range(num_individual)]        
            self.pop = list()
            for [p1, p2] in process_map(self.apply_operations, range(0, num_individual, 2), tqdm_class=tqdm, max_workers=22, chunksize=5):
                self.pop.append(p1)
                self.pop.append(p2)       
            print(best.cost)
            print(best.position)
        return [best.cost, best.position]

# Main

In [6]:
def cost_function(solution):       
    pass

In [ ]:
num_dimensions = len(config["means"]) + 1 + 3

bounds = [            
    *[(-100, 100) for i in range(len(config["means"]) + 1)],
    *[(0.001, 5),(0.001, 1), (0.001, 0.35)] #trigger_coeff, resistance_coeff, buy_each_time
    ]
#PSO(cost_function, bounds, num_particles=1000, maxiter=50, num_dimensions=num_dimensions)

r_cross = 0.9
r_mut = 1.0 / float(num_dimensions)
r_mut /= 2
Genetic_Algorithm(costFunc=cost_function,
                  bounds=bounds, 
                  num_individual=5000,
                  maxiter=50,
                  num_dimensions=num_dimensions, 
                  r_cross=r_cross,
                  r_mut=r_mut)